# Première partie
## -**Nettoyage des données**


In [87]:
import pandas as pd
import numpy as np

In [88]:
full_df = pd.read_csv('data/lol_worlds_2019.csv', na_values = ' ').drop(['url', 'league', 'split', 'date', 'patchno', 'playerid'], axis = 1).set_index('gameid', drop = False)
full_df.earnedgoldshare = np.round(full_df.earnedgoldshare * 100, decimals = 2)
full_df.wardshare = np.round(full_df.wardshare * 100, decimals = 2)
full_df.dmgshare = np.round(full_df.dmgshare * 100, decimals = 2)
full_df['epicmonsterkills'] = full_df.teamdragkills + full_df.teambaronkills + full_df.herald

In [103]:
team_df = full_df[full_df.position == "Team"]
team_df = team_df.dropna(axis = 1, how = 'all')

players_df = full_df[full_df.position != "Team"]
players_df = players_df.dropna(axis = 1, how = 'all')
players_df['kda'] = (players_df.k + players_df.a) / players_df.d.replace(0, 1)

nb_games = team_df[team_df.result == 1].shape[0]
print(nb_games)

119


**Statistiques globales**

In [90]:
gamelen_df = team_df[['gamelength']][team_df.result == 1]

gold_df = team_df[['gdat10', 'gdat15']][team_df.result == 1]
gold_df['gdatend'] = team_df[team_df.result == 1].totalgold - team_df[team_df.result == 0].totalgold

cs_df = players_df[['position', 'csat10', 'csat15']]
csd_df = players_df[['position','csdat10', 'csdat15']][players_df.result == 1]
xpd_df = players_df[['position','xpdat10']][players_df.result == 1]

stats_df = players_df[['position', 'dmgshare', 'earnedgoldshare', 'earnedgpm', 'wardshare', 'kda']]

**Champions**

In [91]:
bans_df = team_df.melt(id_vars= 'gameid',value_vars=['ban1','ban2', 'ban3', 'ban4', 'ban5']).drop('variable', axis = 1).set_index('gameid').rename(columns={'value': 'bans'})
pick_df = players_df[['result','champion']]

**Side**

In [92]:
redside_df = team_df[['result', 'gdat10', 'gdat15', 'epicmonsterkills']][team_df.side == 'Red']
blueside_df = team_df[['result', 'gdat10', 'gdat15', 'epicmonsterkills']][team_df.side == 'Blue']

**Objectifs**

In [93]:
drakes_df = team_df[['result','teamdragkills', 'fd','fdtime', 'firedrakes', 'waterdrakes','earthdrakes', 'airdrakes']]
herald_df = team_df[['result','herald', 'heraldtime']]
baron_df = team_df[['result', 'teambaronkills', 'fbaron','fbarontime']]
elder_df = team_df[['result', 'elders']]
turrets_df = team_df[['result','ft', 'fttime','teamtowerkills']]

# Deuxième partie
## -**Analyse des données**

**Longueur de partie moyenne, partie la plus longue, partie la plus courte.**

In [94]:
gamelen_mean = int(np.round(gamelen_df.to_numpy().mean()))
gamelen_max = int(np.round(gamelen_df.to_numpy().max()))
gamelen_min = int(np.round(gamelen_df.to_numpy().min()))
print('La longueur de partie a été en moyenne de {} minutes,\nla partie la plus longue et la plus courte ont duré respectivement de {} et {} minutes'.format(gamelen_mean, gamelen_max, gamelen_min))

La longueur de partie a été en moyenne de 33 minutes,
la partie la plus longue et la plus courte ont duré respectivement de 63 et 23 minutes


**Gold diff (10 min, 15 min, fin de la partie).**

In [95]:
gdat10_mean = int(np.round(gold_df.gdat10.mean()))
gdat15_mean = int(np.round(gold_df.gdat15.mean()))
gdatend_mean = int(np.round(gold_df.gdatend.mean()))
print('Moyenne des différences de golds à 10 min {}g, 15 min {}g et à la fin de partie {}g pour les équipes gagnantes.'.format(gdat10_mean, gdat15_mean, gdatend_mean))

Moyenne des différences de golds à 10 min 536g, 15 min 1598g et à la fin de partie 9712g pour les équipes gagnantes.


**Creep score (10 min , 15 min), CS diff / poste**

In [96]:
def getMeanPerPos(df, column):
    top_mean = int(np.round(df[df.position == 'Top'][column].mean()))
    jgl_mean = int(np.round(df[df.position == 'Jungle'][column].mean()))
    mid_mean = int(np.round(df[df.position == 'Middle'][column].mean()))
    bot_mean = int(np.round(df[df.position == 'ADC'][column].mean()))
    sup_mean = int(np.round(df[df.position == 'Support'][column].mean()))
    return top_mean, jgl_mean, mid_mean, bot_mean, sup_mean

#csat10
top,jgl,mid,bot,sup = getMeanPerPos(cs_df, 'csat10')
print('Moyenne des Creep score à 10 min :\n-Top {}cs\n-Jungle {}cs\n-Middle {}cs\n-Bot {}cs\n-Support {}cs\n'.format(top,jgl,mid,bot,sup))

#csat15
top,jgl,mid,bot,sup = getMeanPerPos(cs_df, 'csat15')
print('Moyenne des Creep score à 15 min :\n-Top {}cs\n-Jungle {}cs\n-Middle {}cs\n-Bot {}cs\n-Support {}cs\n'.format(top,jgl,mid,bot,sup))

#csdat10
top,jgl,mid,bot,sup = getMeanPerPos(csd_df, 'csdat10')
print('Moyenne des différences de Creep score à 10 min :\n-Top {}cs\n-Jungle {}cs\n-Middle {}cs\n-Bot {}cs\n-Support {}cs\n'.format(top,jgl,mid,bot,sup))

#csdat15
top,jgl,mid,bot,sup = getMeanPerPos(csd_df, 'csdat15')
print('Moyenne des différences de Creep score à 15 min :\n-Top {}cs\n-Jungle {}cs\n-Middle {}cs\n-Bot {}cs\n-Support {}cs'.format(top,jgl,mid,bot,sup))

Moyenne des Creep score à 10 min :
-Top 75cs
-Jungle 57cs
-Middle 86cs
-Bot 84cs
-Support 9cs

Moyenne des Creep score à 15 min :
-Top 120cs
-Jungle 86cs
-Middle 137cs
-Bot 134cs
-Support 16cs

Moyenne des différences de Creep score à 10 min :
-Top 1cs
-Jungle 0cs
-Middle 4cs
-Bot 2cs
-Support -1cs

Moyenne des différences de Creep score à 15 min :
-Top 2cs
-Jungle 1cs
-Middle 7cs
-Bot 5cs
-Support -1cs


**Xp(10 min) diff / poste.**

In [97]:
#xpdat10
top,jgl,mid,bot,sup = getMeanPerPos(xpd_df, 'xpdat10')
print('Moyenne des différences en expérience à 10 min :\n-Top {}\n-Jungle {}\n-Middle {}\n-Bot {}\n-Support {}'.format(top,jgl,mid,bot,sup))

Moyenne des différences en expérience à 10 min :
-Top 56
-Jungle 57
-Middle 144
-Bot 80
-Support 5


**Stats (KDA, Dmg share, Gold share, Golds/minutes, Wards share) / poste.**

In [99]:
#kda
top,jgl,mid,bot,sup = getMeanPerPos(stats_df, 'kda')
print('Moyenne du kda :\n-Top {}\n-Jungle {}\n-Middle {}\n-Bot {}\n-Support {}\n'.format(top,jgl,mid,bot,sup))

#Dmg share
top,jgl,mid,bot,sup = getMeanPerPos(stats_df, 'dmgshare')
print('Moyenne de la répartition des dégats:\n-Top {}%\n-Jungle {}%\n-Middle {}%\n-Bot {}%\n-Support {}%\n'.format(top,jgl,mid,bot,sup))

#Gold share
top,jgl,mid,bot,sup = getMeanPerPos(stats_df, 'earnedgoldshare')
print('Moyenne de la répartition des golds:\n-Top {}%\n-Jungle {}%\n-Middle {}%\n-Bot {}%\n-Support {}%\n'.format(top,jgl,mid,bot,sup))

#gpm
top,jgl,mid,bot,sup = getMeanPerPos(stats_df, 'earnedgpm')
print('Moyenne de golds par minutes:\n-Top {}/min\n-Jungle {}/min\n-Middle {}/min\n-Bot {}/min\n-Support {}/min\n'.format(top,jgl,mid,bot,sup))

#Ward share
top,jgl,mid,bot,sup = getMeanPerPos(stats_df, 'wardshare')
print('Moyenne de la répartition des wards:\n-Top {}%\n-Jungle {}%\n-Middle {}%\n-Bot {}%\n-Support {}%\n'.format(top,jgl,mid,bot,sup))

Moyenne du kda :
-Top 4
-Jungle 5
-Middle 5
-Bot 5
-Support 4

Moyenne de la répartition des dégats:
-Top 25%
-Jungle 16%
-Middle 25%
-Bot 25%
-Support 9%

Moyenne de la répartition des golds:
-Top 23%
-Jungle 17%
-Middle 23%
-Bot 27%
-Support 10%

Moyenne de golds par minutes:
-Top 260/min
-Jungle 196/min
-Middle 267/min
-Bot 309/min
-Support 120/min

Moyenne de la répartition des wards:
-Top 13%
-Jungle 21%
-Middle 12%
-Bot 11%
-Support 43%



**Les champions les plus Picks/Bans.**

In [114]:
display(bans_df.bans.value_counts().head())
display(pick_df[pick_df.champion == 'Pantheon'])

Pantheon    118
Qiyana       86
Syndra       66
Renekton     61
Akali        54
Name: bans, dtype: int64

,result,champion
gameid,,
1070457,1,Pantheon
